# Titanic_RandomForest

어떤 특징(feature)을 갖는 승객이 침몰하는 배에서 생존했는지를 분석하고,
새로운 승객 정보가 주어졌을 때 생존할 수 있는지를 예측하는 kaggle competition의 기본 과제

1. train.csv:
train.csv는 탑승한 승객들 정보를 .csv파일 형태로 가지고 있음. <br>
(엑셀 파일 형태로 정리되어 있다고 생각) <br>
총 891명의 승객 정보를 가지고 있음
- Survied column이 target column -> 우리가 예측해야하는 특징값 (feature)
    - "1"이면 생존, "0"이면 사망

2. test.csv:
test.csv 역시 탑승한 승객들 정보를 .csv 파일 형태로 가지고 있음. <br>
(엑셀 파일 형태로 정리되어 있다고 생각) <br>
총 418명의 승객 정보를 가지고 있음.
- test.csv에는 "Survived" column이 없음 -> 검사할 때 추가해야 함

3. gender_submission.csv:
예측 csv파일을 어떻게 만들어야 하는지 예시를 보여줌
- submission file에는 "PassengerId"랑 "Survived" column이 있어야함

### Data download
1. `pip install kaggle` -> 오류 발생 시 `pip install --user kaggle`

2. API Key 발급

API credentials
To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials. Place this file in the location ***~/.kaggle/kaggle.json***. You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to $KAGGLE_CONFIG_DIR/kaggle.json

For your security, ensure that other users of your computer do not have read access to your credentials. On Unix-based systems you can do this with the following command:

`chmod 600 ~/.kaggle/kaggle.json`

- 혹은 아래 블로그 참고
https://polarcompass.tistory.com/147

설치 완료 후 아래 명령어 수행<br>
`kaggle competitions download -c titanic`

### import pakages

In [12]:
# numpy : 행렬이나 다차원 배열을 쉽게 처리할 수 있도록 지원하는 파이썬의 라이브러리 -> 수치 계산을 효율적으로 하기 위해 불러옴
# pandas : 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 테이블을 만들기 위해 불러옴

import numpy as np # numpy를 np로 나타내기
import pandas as pd # pandas를 pd로 나타내기
import os

# data 파일에 train, test csv 파일이 있는지를 확인 -> 경로를 출력
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/test.csv
./data/train.csv
./data/gender_submission.csv


### load dataset

In [15]:
train_data = pd.read_csv("./data/train.csv") # train에 대한 정보를 pd 패키지를 사용해서 train_data라는 변수에 저장
train_data.head() # csv파일의 위에 5개 요소 출력 # ()안에 숫자 만큼 요소 출력 (defult 5개)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("./data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [21]:
# train_data에서 female 인 사람들만 추출하고 그 중 "Survived" column(행)값만 가져옴
women = train_data.loc[train_data.Sex == 'female']["Survived"]
women # 가져온 값을 출력한다

1      1
2      1
3      1
8      1
9      1
      ..
880    1
882    0
885    0
887    1
888    0
Name: Survived, Length: 314, dtype: int64

In [6]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]

# sum(women) -> 살아남았으면 1이고, 아니면 0이니까 survived column값을 다 더하면 살아남은 여성들의 수를 구할 수 있음 
# len(women) -> women만 추출된 데이터의 길이
rate_women = sum(women)/len(women)

# 살아남은 여성의 비율을 출력
print("% of women who survived: ", rate_women)

% of women who survived:  0.7420382165605095


In [7]:
men = train_data.loc[train_data.Sex=='male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived: ", rate_men)

% of men who survived:  0.18890814558058924


### 패턴 분석
- 약 74퍼센트의 여성이 생존, 19프로의 남성이 사망
- 입력 데이터가 여성이면 26퍼센트의 오류 발생 가능성, 남성이면 18% 오류 발생 가능성 존재
- gender_based submission은 오직 `Sex`라는 하나의 feature colum (특징값)만 고려해서 생존여부 판단
- 여러 개의 column(승객의 특징)들을 결합하여 더 복잡한 패턴을 발견해서 더 좋은 예측으로 이끌어야 한다.

### Use Random forest model 
- Random forest model
    - 분류 및 회귀 작업에 사용되는 기계 학습 알고리즘
    - 일련의 규칙에 따라 의견을 제시하는 간단한 모델인 의사 결정 트리 개념을 기반으로 한다.
    - 학습데이터를 과하게 학습하기 쉬워서, 훈련 데이터에서는 잘 수행되지만, 새 데이터에서는 일반화되지 않을 수 있다.
- example
    - question 1 : 승객이 여자인가 남자인가?
        - question 2-1 : 승객이 여자라면 1등석에 탔는가, 2등석에 탔는가 ...
            - question 3-1-1 : 여자인데 1등석에 탔다면 동승자가 1명인가, 2명인가 ...
    - 승객이 여자고, 1등석에 탔고, 동승자가 1명이고, ... , 는 대체로 죽었다. 라는 패턴 파악 -> 이를 근거로 판단
- sklearn.ensemble 에서 RandomForestClassifier 모델을 import 해서 사용

In [8]:
# train_data 에서 원하는 column을 가지고 온다
train_data[["Pclass", "Sex", "SibSp", "Parch"]]

,Pclass,Sex,SibSp,Parch
0,3,male,1,0
1,1,female,1,0
2,3,female,0,0
3,1,female,1,0
4,3,male,0,0
...,...,...,...,...
886,2,male,0,0
887,1,female,0,0
888,3,female,1,2
889,1,male,0,0


### 데이터 전처리
- 사용할 데이터 (input data)를 선택하여 분석에 알맞은 형태로 가공하는 일
- 데이터 전처리에는
    - 데이터 클렌징
    - 결손값 처리(채우기와 채워넣기)
    - 데이터 인코딩
    - 이상치 제거
    - feature 선택, 추출, 가공 등

### one - hot encoding

- 머신러닝은 문자형을 인식하지 못하기 때문에 수치형으로 변환해주어야 한다
- 예를들어 해, 달, 별 같이 숫자로 표현할 수 없게 표현되어 있는 데이터를 숫자로 나타내줌
    - 0, 1, 2에 이미 수치적 의미가 있기 때문에,
    - 해 = (1, 0, 0, ...), 달 = (0, 1, 0, ...), 별 = (0, 0, 1, ...) 이런 식으로 표현 가능
- 수행하면 각 카테고리에 해당하는 변수들이 모두 column에 더해지게 된다. 따라서 카테고리가 너무 많은 경우에는 사용하기 적합하지 않음


In [22]:
# RandomForest package import 
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"] # y에 생존여부(0,1) 추출

# 4개의 특성 (객실 등급, 성별, 형제 or 배우자 수, 부모 or 자녀 수)만 패턴 파악에 고려
features = ["Pclass", "Sex", "SibSp", "Parch"] 

# pd.get_dummies: 모든 데이터를 수치로 변환해주는 전처리 작업
# one hot encoding
X = pd.get_dummies(train_data[features]) # X에 전처리한 train data의 데이터를 추출 

X_test = pd.get_dummies(test_data[features]) # test data로 train data와 같은 전처리를 가짐

# 모델 객체 생성
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

#.fit()함수를 사용해서 학습
model.fit(X, y)

# .fit을 통해 훈련된 model을 가지고 predict
predictions = model.predict(X_test)

# 제출할 수 있도록 "PassengerId" column과 "Survived" column만 남긴 Dataframe을 만듬
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

# 이걸 .csv 파일로 생성 -> 사이트에 제출하기 위함
output.to_csv('submission.csv', index=False)

print("Your submission was successfully saved!")

Your submission was successfully saved!


1. test_data -> 전처리 -> X_test
2. X_test -> 모델 학습 -> prediction
3. test_data에서 passengerID만 가져와서, prediction과 합친다

### Submit
- -c option: competition name
- -f option: submission csv file name
- -m option: submission description (설명글)

In [ ]:
!kaggle competitions submit -c titanic -f submission.csv -m "tutorial"
# !는 터미널 실행 명령어

100%|██████████████████████████████████████| 2.77k/2.77k [00:00<00:00, 18.2kB/s]

### view results

In [2]:
!kaggle competitions submissions -c titanic

fileName        date                 description  status    publicScore  privateScore  
--------------  -------------------  -----------  --------  -----------  ------------  
submission.csv  2023-05-08 13:05:58  tutorial     complete  0.77511                    
submission.csv  2023-05-08 12:59:22  tutorial     complete  0.77511                    
submission.csv  2023-05-08 12:54:21  tutorial     complete  0.77511                    
submission.csv  2023-05-08 11:40:29  tutorial     complete  0.77511                    
